# テキスト操作の比較
## ライブラリ準備

In [1]:
import logging
import time
import json
import os

from dotenv import load_dotenv
import requests

import cohere
import anthropic
import openai
from openai import OpenAI

In [2]:
load_dotenv()

CLAUDE_API_KEY = os.getenv("CLAUDE_APIKEY")
COHERE_API_KEY = os.getenv("COHERE_APIKEY")
OPENAI_API_KEY = os.getenv("OPENAI_APIKEY")

## 各サービスへのクライアントの定義

In [7]:
translation_query = lambda text: f"""以下の英語テキストを日本語に翻訳してください。
翻訳は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
英語テキスト:
{text}

日本語翻訳:"""

summarization_query = lambda text: f"""以下のテキストを日本語で 400 字程度に要約してください。
翻訳は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
オリジナルテキスト:
{text}

日本語要約:"""

class AIModelClient:

    def translate(self, text: str) -> Dict:
        start_time = time.time()
    
        try:
            translation, model = self._concrete_translate(text)
            elapsed_time = time.time() - start_time
            
            return {
                "translation": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "translation": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }

    def summarize(self, text: str) -> Dict:
        start_time = time.time()
    
        try:
            translation, model = self._concrete_summarize(text)
            elapsed_time = time.time() - start_time
            
            return {
                "translation": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "translation": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }


class ClaudeClient(AIModelClient):
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "claude-sonnet-4-5") -> Dict:
        message = self.client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ]
        )        
        return message.content[0].text, model

    def _concrete_summarize(self, text: str, model: str = "claude-sonnet-4-5") -> Dict:
            message = self.client.messages.create(
                model=model,
                max_tokens=4096,
                messages=[
                    {
                        "role": "user",
                        "content": summarization_query(text)
                    }
                ]
            )            
            return message.content[0].text, model

claude_client = ClaudeClient(CLAUDE_API_KEY)

class CohereClient(AIModelClient):
    def __init__(self, api_key):
        self.client = cohere.ClientV2(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "command-a-translate-08-2025") -> Dict:
        response = self.client.chat(
            model=model,
            message=translation_query(text),
            temperature=0.3
        )
        return response.text, model
    
    def _concrete_summarize(self, text: str, model: str = "command-a-03-2025") -> Dict:
        response = self.client.chat(
            model=model,
            message=summarization_query(text),
            temperature=0.3
        )            
        return response.text, model

    # https://docs.cohere.com/docs/summarizing-text
    def _concrete_summarize(self, text: str, model: str = "command-a-03-2025") -> Dict:
        response = self.client.chat(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "与えられたテキストを日本語で 400 字程度に要約してください。"
                },
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ],
            temperature=0.3
        )            
        return response.message.content[0].text, model

cohere_client = CohereClient(COHERE_API_KEY)

In [4]:
class OpenAIClient:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "gpt-4o") -> Dict:
        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な翻訳者です。英語を自然で流暢な日本語に翻訳してください。"
                },
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ],
            temperature=0.3
        )        
        return response.choices[0].message.content, model

    def _concrete_summarize(self, text: str, model: str = "gpt-4o") -> Dict:
        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な編集者です。自然で流暢な日本語に要約してください。"
                },
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ],
            temperature=0.3
        )            
        return response.choices[0].message.content, model

openai_client = OpenAIClient(OPENAI_API_KEY)

In [5]:
# Project Gutenberg, https://www.gutenberg.org/ebooks/12096
TEXT_URL = 'https://www.gutenberg.org/cache/epub/12096/pg12096.txt'
TEXT_FILE = 'data/I.Nitobe_Bushido.txt'
if not os.path.exists(TEXT_FILE):
    res = requests.get(TEXT_URL)
    with open(TEXT_FILE, "w") as f:
        f.write(res.text)
    
with open(TEXT_FILE) as f:
    text = f.read()

print(f"text size: {len(text)}")

text size: 217583


In [6]:
print(json.dumps(claude_client.summarize(text), indent=2, ensure_ascii=False))
print(json.dumps(cohere_client.summarize(text), indent=2, ensure_ascii=False))
#print(openai_client.summarize(text))

{
  "translation": "# 『武士道』要約(約400字)\n\n新渡戸稲造著『武士道、日本の魂』は、日本の武士階級の倫理規範を西洋に紹介した著作である。武士道は成文化されない道徳律で、封建制度の中で数世紀かけて形成された。その源泉は神道、仏教、儒教にあり、特に孔子と孟子の教えが大きな影響を与えた。\n\n武士道の主要な徳目として、義(正義)、勇(勇気)、仁(慈悲)、礼(礼儀)、誠(誠実)、名誉、忠義が挙げられる。これらは単なる観念ではなく、実践を通じて人格形成の基礎となった。武士の教育は知識よりも人格の陶冶を重視し、剣術、弓術、柔術などの武芸と共に、書道や詩歌も学んだ。\n\n切腹と敵討ちという特殊な制度は、名誉を最高の価値とする武士道精神の表れであった。刀は武士の魂の象徴とされ、常に身につけられた。女性も武士道の影響を受け、貞節と自己犠牲の精神を重んじた。\n\n武士道の影響は武士階級にとどまらず、庶民にまで浸透し、日本の国民性を形成した。近代化と共に武士道は衰退しつつあるが、その精神は日本人の心に生き続けており、新しい時代においても道徳的指針として意義を持ち続けるであろう。",
  "time": 13.983185052871704,
  "model": "claude-sonnet-4-5",
  "success": true,
  "error": null
}
{
  "translation": "『武士道』は、新渡戸稲造による日本の武士道の精神を解説した書籍である。新渡戸は、武士道が日本の道徳的価値観の基盤であり、忠義、勇気、仁義、礼節、誠実さなどの美徳を重視していると説明する。武士道は、封建制度下で発展した武士階級の倫理規範であり、彼らの日常生活や行動指針となった。\n\n新渡戸は、武士道の源流として、仏教の運命受け入れの精神、神道の神々や先祖への敬意、儒教の五常の教えを挙げる。特に儒教の「仁」の思想は、武士の統治者としての責任感や民衆への思いやりを育んだ。\n\n武士道は、正義、勇気、仁愛、礼節、誠実さなどの徳目を重んじる。正義は武士の行動規範の核心であり、勇気は正しいことを行うための精神的強さとされる。仁愛は弱者への思いやりであり、礼節は他者への配慮から生まれる。誠実さは武士の言葉の重みを表し、約束は口頭でも厳守された。\n

※この Notebook は、API の使い方を試す目的のものです。 AI の出力は不正確な場合があります。※